In [10]:
import numpy as np
import lasagne
from nbfinder import NotebookFinder
import sys
sys.meta_path.append(NotebookFinder())


from helper_fxns import iterate_minibatches, early_stop, plot_training
from build_hur_classif_network import build_network

importing Jupyter notebook from helper_fxns.ipynb
importing Jupyter notebook from build_hur_classif_network.ipynb


In [9]:
def train_one_epoch(x,y,num_minibatches, train_fn, val_fn):
    err = 0
    acc = 0
    batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(x, y, num_minibatches, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        _, acc = val_fn(inputs, targets)
        train_acc += acc
        train_batches += 1
    return err, acc, batches

In [6]:
def val_one_epoch(x,y,num_miniBatches, val_fn):
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_val, y_val, 50, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1
        return val_err, val_acc, val_batches

In [7]:
def print_results(epoch, num_epochs, start_time, tr_err, tr_acc, val_err, val_acc):
    # Then we print the results for this epoch:
    print "Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time)
    print "  training loss:\t\t{:.6f}".format(tr_err)
    print "  training acc:\t\t{:.6f}".format(tr_acc)
    print "  validation loss:\t\t{:.6f}".format(val_err)
    print "  validation accuracy:\t\t{:.2f} %".format(val_acc)

In [8]:
def train(datasets, num_epochs, save=False, frac_of_datasets=1, network_kwargs={}):
    train_fn,val_fn,network = build_network(**network_kwargs)
    print "Starting training..." 
    X_train, y_train, X_val, y_val, X_test, y_test = datasets

    train_errs, train_accs, val_errs, val_accs = 4*([])
    
    for epoch in range(num_epochs):
        tr_err, tr_acc, tr_batches = train_one_epoch(X_train, y_train,
                                                     num_minibatches=1000,
                                                     train_fn=train_fn,
                                                     val_fn=val_fn)
        
        val_err, val_acc, val_batches = val_one_epoch(X_val, y_val,
                                                     num_minibatches=50,
                                                      val_fn=val_fn)

        for seq, num, scal in zip([train_errs, train_accs, val_errs, val_accs],
                             [tr_err, tr_acc, val_err, val_acc], 
                             [tr_batches, tr_batches, val_batches, val_batches] ):
            seq.append(num / scal )


        print_results(epoch, num_epochs, start_time, tr_err, tr_acc, val_err, val_acc)
        plot_training(train_errs,[], 'err')
        


    if save and epoch % 10 == 0:
    # Optionally, you could now dump the network weights to a file like this:
        np.savez('model.npz', *lasagne.layers.get_all_param_values(network))